In [1]:
import pandas as pd

CSV_FILES = [
    "Huawei_inverter.csv", 
    "Azzuro_inverter.csv"]


dfs = []
for csv_file in CSV_FILES:
    df = pd.read_csv(csv_file)
    dfs.append(df)
 
df = pd.concat(dfs, ignore_index=True)
df

,ModelName,Vac,Pso,Iac,Paco,Pdco,Vdco,C0,C1,C2,C3,Pnt,Vdcmax,Idcmax,Mppt_low,Mppt_high,number_mppts,Isc_mppt,max_strings
0,SUN2000-30KTL-M3,230.0,NaN,43.3,33000.0,NaN,600.0,NaN,NaN,NaN,NaN,5.5,1100,27/20,200,1000,4,40,8
1,SUN2000-36KTL-M3,400.0,NaN,52.0,40000.0,NaN,600.0,NaN,NaN,NaN,NaN,5.5,1100,27/20,200,1000,4,40,8
2,SUN2000-40KTL-M3,400.0,NaN,57.8,44000.0,NaN,600.0,NaN,NaN,NaN,NaN,5.5,1100,27/20,200,1000,4,40,8
3,SUN2000-115KTL-M2,400.0,NaN,182.3,125000.0,NaN,600.0,NaN,NaN,NaN,NaN,3.5,1100,30/20,200,1000,8,40,2
4,SUN2000-30K-MC0,400.0,NaN,43.3,33000.0,NaN,600.0,NaN,NaN,NaN,NaN,NaN,1100,27/23,180,1000,4,40,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,1PH HYD 3600 ZSS HP,NaN,5400.0,3680.0,16.0,5400.0,360,NaN,NaN,NaN,NaN,NaN,600,16A/16A,90,550,2,22.5A/22.5A,1
127,1PH HYD 4000 ZSS HP,NaN,6000.0,4400.0,20.0,6000.0,360,NaN,NaN,NaN,NaN,NaN,600,16A/16A,90,550,2,22.5A/22.5A,1
128,1PH HYD 4600 ZSS HP,NaN,6900.0,4600.0,20.9,6900.0,360,NaN,NaN,NaN,NaN,NaN,600,16A/16A,90,550,2,22.5A/22.5A,1
129,1PH HYD 5000 ZSS HP,NaN,7500.0,5500.0,25.0,7500.0,360,NaN,NaN,NaN,NaN,NaN,600,16A/16A,90,550,2,22.5A/22.5A,1


In [2]:
import uuid
import pandas as pd


def num_or_null(val):
    return "NULL" if pd.isna(val) else val

def int_or_null(val):
    return "NULL" if pd.isna(val) else int(val)

def str_or_null(val):
    return "NULL" if pd.isna(val) else f"'{val}'"

sql_statements = []
ids = [] 
for _, row in df.iterrows():
    new_id = uuid.uuid4()
    ids.append(new_id)
    sql = f"""
    INSERT INTO inverter_technology (
        id,
        cec_name,
        nominal_ac_voltage,
        dc_power_start,
        nominal_ac_power,
        dc_power_ref,
        dc_voltage_ref,
        coeff_c0,
        coeff_c1,
        coeff_c2,
        coeff_c3,
        night_tare_loss,
        max_dc_voltage,
        max_dc_current,
        mppt_low_voltage,
        mppt_high_voltage
    ) VALUES (
        '{new_id}',
        {str_or_null(row["ModelName"])},
        {num_or_null(row["Vac"])},
        {num_or_null(row["Pso"])},
        {num_or_null(row["Paco"])},
        {num_or_null(row["Pdco"])},
        {num_or_null(row["Vdco"])},
        {num_or_null(row["C0"])},
        {num_or_null(row["C1"])},
        {num_or_null(row["C2"])},
        {num_or_null(row["C3"])},
        {num_or_null(row["Pnt"])},
        {num_or_null(row["Vdcmax"])},
        {str_or_null(row["Idcmax"])},
        {num_or_null(row["Mppt_low"])},
        {num_or_null(row["Mppt_high"])}
    );
    """
    sql_statements.append(sql)

with open("seed_inverter_module_technology_inserts.sql", "w") as f:
    f.write("\n".join(sql_statements))

delete_sql = f"""
    DELETE FROM pv_module_technology
    WHERE id IN ({', '.join(f"'{i}'" for i in ids)});
    """

with open("seed_inverter_module_technology_delete.sql", "w") as f:
    f.write(delete_sql)

print("SQL files generated")


SQL files generated
